In [2]:
import mindspore
import  sys
import  time
import  glob
import  numpy as np
import  logging
import  argparse
import os

from mindspore.train.callback import ModelCheckpoint, CheckpointConfig, LossMonitor,TimeMonitor
from mindspore import Tensor, Model

from mindspore import context, DatasetHelper, connect_network_with_dataset
from mindspore import dtype as mstype
from mindspore import nn,DynamicLossScaleManager
from mindspore.parallel._utils import _get_device_num

from mindspore.common import set_seed
from mindspore.context import ParallelMode
from mindspore.communication.management import init, get_rank, get_group_size
from nlr import NLRNet
from mindspore import dtype as mstype
from loss import do_Loss

from data import get_dataset


In [3]:
context.set_context(mode=context.PYNATIVE_MODE, device_target="GPU")

In [4]:
random_seed = 1996
np.random.seed(random_seed)

In [6]:
net = NLRNet()
mindspore.load_checkpoint('/code/uldr-23_700.ckpt',net)
net.set_train(False)
data = get_dataset(train='test')
batch_num = data.train_dataset.get_dataset_size()
#rank = set_device(args)


/dataset/WorldView2_reduce/reduce
record_1.mat
record_4.mat
record_3.mat
record_8.mat
record_6.mat
record_5.mat
record_9.mat
record_7.mat
16 576


In [5]:
data.train_dataset.get_dataset_size() 

36

In [7]:
576%16

0

In [5]:
it = data.train_dataset.create_dict_iterator()

In [6]:
data_ = next(it)

In [7]:
len(data_)

5

In [21]:
key = list(data_.keys())

In [22]:
key[0]

'input_images0'

In [7]:
from tools import sam, ergas, scc, D_lambda, D_s, qindex

In [8]:
def process_output(collect_output, collect_label, vol_nums):
    # remove rest output
    if vol_nums != 0:

        collect_output[-1] = collect_output[-1][-vol_nums:, :, :, :]
        collect_label[-1] = collect_label[-1][-vol_nums:, :, :, :]

    # condat all batchsize
    collect_output = np.row_stack(np.array(collect_output))
    collect_label = np.row_stack(np.array(collect_label))

    # slice net output
    collect_output[collect_output < 0] = 0
    collect_output[collect_output > 1] = 1

    return collect_output, collect_label

In [9]:
def ref_assement(collect_output, collect_label):
    """down_resolution index test"""
    h, w, c = collect_output[0].shape
    element_nums = h * w * c
    _sam, _erags, _scc, _qn = [], [], [], []
    for item in range(collect_output.shape[0]):
        label_block = collect_label[item, :, :, :]
        output_block = collect_output[item, :, :, :]

        if len(label_block[label_block == 0]) / element_nums > 0.5:
            continue

        _sam.append(sam(label_block, output_block))
        _erags.append(ergas(output_block, label_block))
        _scc.append(scc(output_block, label_block))
        _qn.append(qindex(output_block, label_block, block_size=128))
    return _sam,_erags,_scc,_qn

In [10]:
collect_output, collect_label = [], []
for data_ in data.train_dataset.create_dict_iterator():
    keys = list(data_.keys())
    out = net(data_[keys[0]],data_[keys[1]],data_[keys[2]],data_[keys[3]])
    out_numpy = out.transpose(0,2,3,1).asnumpy()
    label_numpy = data_[keys[4]].transpose(0,2,3,1).asnumpy()
    collect_output.append(out_numpy)
    collect_label.append(label_numpy)
output, label = process_output(collect_output, collect_label, vol_nums = 16)
sam,erags,scc,qn = ref_assement(output, label)

In [15]:
np.array(sam).mean()

0.11348307279760134

In [16]:
np.array(qn).mean()

0.7185041177639572

In [11]:
np.array(erags).mean()

2.2912718881350114